# Neo4J: Example of fictious bank clients

The following code templates have been accessed from: https://neo4j.com/docs/python-manual/current/

In [10]:
from neo4j import GraphDatabase
import pandas as pd

## Connect to the database

In [11]:
# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687"

# --- Run locally: ---
# Install the required packages (requirements.txt)
# Start Docker Desktop and create an image, e.g. through Anaconda prompt:
# $ docker run -p7474:7474 -p7687:7687 -d -e NEO4J_AUTH=neo4j/secretgraph neo4j:latest
# To access your graph over Neo4J Desktop: Start Neo4J Desktop --> Graph Apps (left navigation) --> Neo4j Browser --> type in the credentials (neo4j/secretgraph) --> connect --> bolt://localhost:7687
# URI = "neo4j://localhost"

AUTH = ("neo4j", "secretgraph")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Writing manually into Neo4J

In [12]:
summary = driver.execute_query(
    "CREATE (:Person {name: $name})",
    name="Alice",
    database_="neo4j",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 1 nodes in 1 ms.


C:\Users\pell\AppData\Local\Temp\ipykernel_23212\2429642574.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query(


## Writing manually to the database in Neo4j Browser:

CREATE (:Person {name: 'Peter'})

## Querying

In [13]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

{'name': 'Alice'}
{'name': 'Peter'}
{'name': 'Alice'}
The query `MATCH (p:Person) RETURN p.name AS name` returned 3 records in 0 ms.


C:\Users\pell\AppData\Local\Temp\ipykernel_23212\3877331262.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


## Update the database

In [14]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    SET p.age = $age
    """, name="Alice", age=42,
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'properties_set': 2}.


C:\Users\pell\AppData\Local\Temp\ipykernel_23212\3817737189.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


In [15]:
records, summary, keys = driver.execute_query("""
    MATCH (alice:Person {name: $name})
    MATCH (bob:Person {name: $friend})
    CREATE (alice)-[:KNOWS]->(bob)
    """, name="Alice", friend="Bob",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {}.


C:\Users\pell\AppData\Local\Temp\ipykernel_23212\2454820279.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


## Delete from the database

In [16]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    DETACH DELETE p
    """, name="Alice",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'nodes_deleted': 2}.


C:\Users\pell\AppData\Local\Temp\ipykernel_23212\3368292491.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


## Query parameters

In [17]:
driver.execute_query(
    "MERGE (:Person {name: $name})",
    name="Alice", age=42,
    database_="neo4j",
)

C:\Users\pell\AppData\Local\Temp\ipykernel_23212\926086447.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001E2C3DDB110>, keys=[])

In [18]:
parameters = {
    "name": "Alice",
    "age": 42
}
driver.execute_query(
    "MERGE (:Person {name: $name})",
    parameters_=parameters,
    database_="neo4j",
)

C:\Users\pell\AppData\Local\Temp\ipykernel_23212\1909268578.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001E2C41F7AD0>, keys=[])

## Writing a CSV file into Neo4J

In [19]:
# Connection details

# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687" # Replace with your Neo4j bolt URI

# --- Run locally: ---
#URI = "neo4j://localhost"

username = "neo4j"  # Replace with your Neo4j username
password = "secretgraph"  # Replace with your Neo4j password

# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_clients_and_transactions(tx, clients, transactions):
    # Create client nodes
    for client in clients['client_id']:
        tx.run("MERGE (c:Client {client_id: $client_id})", client_id=client)
    
    # Create transaction relationships (edges)
    for index, row in transactions.iterrows():
        tx.run(
            """
            MATCH (c1:Client {client_id: $client1}), (c2:Client {client_id: $client2})
            MERGE (c1)-[r:TRANSACTED_WITH {transactions: $transactions}]->(c2)
            """, 
            client1=row['client1'], 
            client2=row['client2'], 
            transactions=row['transactions']
        )

# Load CSV data into pandas DataFrame
clients_df = pd.read_csv('workspace/example_data/swiss_bank_clients.csv')  # Update with correct path
transactions_df = pd.read_csv('workspace/example_data/swiss_bank_transactions.csv')  # Update with correct path

# Load data into Neo4j
with driver.session() as session:
    session.write_transaction(create_clients_and_transactions, clients_df, transactions_df)

# Close the driver connection
driver.close()

print("Data loaded into Neo4j successfully!")


NameError: name 'uri' is not defined